# Load Data

In [1]:
import sys
import os

# 현재 파일이 있는 디렉토리 기준으로 상위 폴더 경로 추가
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # 한 단계 상위 폴더
sys.path.append(parent_dir)

print(f"추가된 경로: {parent_dir}")

추가된 경로: /Users/user/Desktop/rec_sys/yamyam-lab


In [2]:
from src.tools.google_drive import ensure_data_files
import pandas as pd

# Ensure required data files are available
data_paths = ensure_data_files()

# Load data into Pandas DataFrames
diner = pd.read_csv(data_paths["diner"])
review = pd.read_csv(data_paths["review"])

# Merge review and reviewer data
review = pd.merge(review, pd.read_csv(data_paths["reviewer"]), on="reviewer_id", how="left")

기존 data가 존재합니다. 파일 경로를 반환합니다.


/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_9492/2730569411.py:8: DtypeWarning: Columns (5,8,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner = pd.read_csv(data_paths["diner"])


In [3]:
data_paths

{'reviewer': '/Users/user/Desktop/rec_sys/yamyam-lab/data/reviewer.csv',
 'category': '/Users/user/Desktop/rec_sys/yamyam-lab/data/diner_category_raw.csv',
 'review': '/Users/user/Desktop/rec_sys/yamyam-lab/data/review.csv',
 'diner': '/Users/user/Desktop/rec_sys/yamyam-lab/data/diner.csv'}

In [4]:
reviewer_df = pd.read_csv(data_paths['reviewer'])
reviewer_df.head(3)

,reviewer_id,reviewer_user_name,reviewer_avg,badge_grade,badge_level
0,1114732494,Poop,2.3,브론즈 배지,5
1,1206802373,까칠한미식가,5.0,실버 배지,17
2,1142286415,쪠,3.2,골드 배지,31


In [13]:
reviewer_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 667044 entries, 0 to 667043
Data columns (total 5 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   reviewer_id         667044 non-null  int64  
 1   reviewer_user_name  664958 non-null  object 
 2   reviewer_avg        667044 non-null  float64
 3   badge_grade         667044 non-null  object 
 4   badge_level         667044 non-null  int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 25.4+ MB


In [60]:
import pandas as pd
import pandera as pa
from pandera import Column, DataFrameSchema


# 데이터 스키마 정의
reviewer_schema = DataFrameSchema({
    "reviewer_id": Column(int, nullable=False, unique=True),
    "reviewer_user_name": Column(str, nullable=True),
    "reviewer_avg": Column(float, nullable=True),
    "badge_grade":  Column(str, nullable=True),
    "badge_level": Column(int, nullable=True)
})

validated_reviewer_df = reviewer_schema.validate(reviewer_df)

In [19]:
category_df = pd.read_csv(data_paths['category'])
category_df.head(3)

,diner_idx,industry_category,diner_category_large,diner_category_middle,diner_category_small,diner_category_detail
0,17369290,음식점,카페,NaN,NaN,NaN
1,699662517,음식점,한식,"육류,고기",닭요리,NaN
2,1390211388,음식점,분식,떡볶이,국대떡볶이,NaN


In [20]:
# 데이터 스키마 정의
category_schema = DataFrameSchema({
    "diner_idx": Column(int, nullable=False, unique=True),
    "industry_category": Column(str, nullable=True),
    "diner_category_large": Column(str, nullable=True),
    "diner_category_middle":  Column(str, nullable=True),
    "diner_category_small": Column(str, nullable=True),
    "diner_category_small": Column(str, nullable=True)
})

validated_category_df = category_schema.validate(category_df)

In [63]:
review_df = pd.read_csv(data_paths['review'])
review_df['diner_idx'] = review_df['diner_idx'].astype(int)
review_df['reviewer_review_date'] = pd.to_datetime(review_df['reviewer_review_date'])

review_df.head(3)

,diner_idx,reviewer_id,review_id,reviewer_review,reviewer_review_date,reviewer_review_score
0,1001596323,256114348,5916807,NaN,2022-07-26,5.0
1,20744478,256114348,843568,NaN,2019-06-26,4.0
2,13320353,256114348,3978448,NaN,2021-07-19,5.0


In [ ]:
# 데이터 스키마 정의
review_schema = DataFrameSchema({
    "diner_idx": Column(int, nullable=False, unique=False),
    "reviewer_id": Column(int, nullable=False, unique=False),
    "review_id": Column(int, nullable=False, unique=True),
    "reviewer_review": Column(str, nullable=True),
    "reviewer_review_date":  Column(pa.DateTime, nullable=True),
    "reviewer_review_score": Column(float, nullable=True)
})

validated_review_df = review_schema.validate(review_df)

In [40]:
diner_df = pd.read_csv(data_paths['diner'])
diner_df.head(3)

/var/folders/bw/_nzx77m90994jql91xkcq2kc0000gn/T/ipykernel_8288/4217478099.py:1: DtypeWarning: Columns (5,8,14) have mixed types. Specify dtype option on import or set low_memory=False.
  diner_df = pd.read_csv(data_paths['diner'])


,diner_idx,diner_name,diner_tag,diner_menu_name,diner_menu_price,diner_review_cnt,diner_review_avg,diner_blog_review_cnt,diner_review_tags,diner_road_address,diner_num_address,diner_phone,diner_lat,diner_lon,diner_open_time,diner_open_time_titles,diner_open_time_hours,diner_open_time_off_days_title,diner_open_time_off_days_hours,bayesian_score
0,17369290,커피소녀,"['디저트카페', '핸드드립', '카카오페이', '제로페이']","['에스프레소솔로', '아메리카노', '카페라떼', '유자차', '로얄밀크티', '...","['3800', '4300', '4800', '5000', '5000', '5000...",68,4.9,3.9,"[('친절', 31), ('맛', 18), ('분위기', 13), ('가성비', 8...",서울 영등포구 의사당대로 38,서울 영등포구 여의도동 17,02-780-1199,37.526082,126.918423,NaN,['영업시간'],"['월~금 08:00 ~ 22:00', '토,일 10:00 ~ 22:00']",['휴무일'],"['설당일, 추석당일, 1월1일']",0.455706
1,699662517,숯돌이네닭갈비구이,NaN,"['숯불양념닭갈비(350g)', '숯불간장닭갈비(350g)', '숯불소금닭갈비(35...","['14000', '14000', '13000', '14000', '14000', ...",19,5.0,3.9,"[('맛', 11), ('친절', 11), ('가성비', 7), ('분위기', 6)...",인천 계양구 양지로 155,인천 계양구 귤현동 504,NaN,37.566954,126.746581,NaN,['금일영업마감'],['매일 15:00 ~ 23:00'],NaN,[],0.441318
2,1390211388,국대떡볶이 분당파크뷰점,"['경기지역화폐', '혼밥']",NaN,NaN,58,4.7,4.2,"[('친절', 34), ('맛', 26), ('가성비', 24), ('분위기', 2...",경기 성남시 분당구 정자일로 248,경기 성남시 분당구 정자동 6,NaN,37.372691,127.107330,NaN,['영업시간'],"['월~토 11:00 ~ 20:30', '월~토 라스트오더 ~ 20:00']",['휴무일\n휴무일\n수정 제안\n이동하기'],"['일요일, 더보기']",0.433312


In [58]:
diner.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233044 entries, 0 to 233043
Data columns (total 20 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   diner_idx                       233044 non-null  int64  
 1   diner_name                      233044 non-null  object 
 2   diner_tag                       126797 non-null  object 
 3   diner_menu_name                 167074 non-null  object 
 4   diner_menu_price                135280 non-null  object 
 5   diner_review_cnt                233044 non-null  object 
 6   diner_review_avg                233044 non-null  float64
 7   diner_blog_review_cnt           233044 non-null  float64
 8   diner_review_tags               106621 non-null  object 
 9   diner_road_address              232077 non-null  object 
 10  diner_num_address               233044 non-null  object 
 11  diner_phone                     161402 non-null  object 
 12  diner_lat       

In [65]:
# 데이터 스키마 정의
diner_schema = DataFrameSchema({
    "diner_idx": Column(int, nullable=False, unique=True),
    "diner_name": Column(str, nullable=True),
    "diner_tag": Column(str, nullable=True),
    "diner_menu_name": Column(str, nullable=True),
    "diner_menu_price": Column(str, nullable=True),
    "diner_review_cnt": Column(int, nullable=True),
    "diner_review_avg": Column(float, nullable=True),
    "diner_blog_review_cnt": Column(float, nullable=True),
    "diner_review_tags": Column(str, nullable=True),
    "diner_road_address": Column(str, nullable=True),
    "diner_num_address": Column(str, nullable=True),
    "diner_phone": Column(str, nullable=True),
    "diner_lat": Column(float, nullable=True),
    "diner_lon": Column(float, nullable=True),
    "diner_open_time": Column(str, nullable=True),
    "diner_open_time": Column(str, nullable=True),
    "diner_open_time_titles": Column(str, nullable=True),
    "diner_open_time_hours": Column(str, nullable=True),
    "diner_open_time_off_days_title": Column(str, nullable=True),
    "diner_open_time_off_days_hours": Column(str, nullable=True),
    "bayesian_score": Column(str, nullable=True),
})

validated_diner_df = diner_schema.validate(diner_df)

SchemaError: expected series 'diner_review_cnt' to have type int64, got object

In [1]:
from src.data_validation import DataValidator

data_validator = DataValidator()

ModuleNotFoundError: No module named 'schemas'

In [7]:
data_validator.validate(df=reviewer_df, name_of_df="reviewer")

,reviewer_id,reviewer_user_name,reviewer_avg,badge_grade,badge_level
0,1114732494,Poop,2.3,브론즈 배지,5
1,1206802373,까칠한미식가,5.0,실버 배지,17
2,1142286415,쪠,3.2,골드 배지,31
3,1174940695,거짓없는리뷰,3.3,골드 배지,56
4,2911445434,riasis,4.8,브론즈 배지,6
...,...,...,...,...,...
667039,982698446,슬리퍼,5.0,브론즈 배지,2
667040,2327470553,🌸,5.0,브론즈 배지,2
667041,263664790,손주영,5.0,브론즈 배지,3
667042,1436049593,이수재,5.0,브론즈 배지,3
